<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.851 · Tipología y ciclo de vida de los datos · PRA1</p>
<p style="margin: 0; text-align:right;">2020-2 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# Práctica 1: Web scraping

 - **https://github.com/sfunesolaria/Pra1WebScraping**

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
class PythonSpider(scrapy.Spider):
    name = 'pythonspider'

    start_urls = ['https://arxiv.org/list/cs.LG/recent/',]
    found_events = []

    def parse(self, response):
        for event in response.xpath('//div[contains(@class, "meta")]'):
            event_details = dict()
            event_details['title'] = event.xpath('div[contains(@class, "list-title mathjax")]/span/following-sibling::text()').extract_first()
            
            # only takes one author
            event_details['authors'] = event.xpath('div[contains(@class, "list-authors")]/a/text()').extract_first()
            
            event_details['comments'] = event.xpath('div[contains(@class, "list-comments mathjax")]/span/following-sibling::text()').extract_first()
            
            event_details['primary-subject'] = event.xpath('div[contains(@class, "list-subjects")]/span[contains(@class, "primary-subject")]/text()').extract_first()
            
            self.found_events.append(event_details)

if __name__ == "__main__":
    process = CrawlerProcess({ 'LOG_LEVEL': 'ERROR'})
    process.crawl(PythonSpider)
    spider = next(iter(process.crawlers)).spider
    process.start()

    for event in spider.found_events: print(event)

{'title': ' PlasticineLab: A Soft-Body Manipulation Benchmark with Differentiable  Physics\n', 'authors': 'Zhiao Huang', 'comments': ' Accepted to ICLR 2021 as a spotlight presentation. Project page: ', 'primary-subject': 'Machine Learning (cs.LG)'}
{'title': ' Regularizing Generative Adversarial Networks under Limited Data\n', 'authors': 'Hung-Yu Tseng', 'comments': ' CVPR 2021. Project Page: ', 'primary-subject': 'Machine Learning (cs.LG)'}
{'title': ' Learned transform compression with optimized entropy encoding\n', 'authors': 'Magda Gregorová', 'comments': ' Neural Compression Workshop @ ICLR 2021\n', 'primary-subject': 'Machine Learning (cs.LG)'}
{'title': ' Modern Hopfield Networks for Few- and Zero-Shot Reaction Prediction\n', 'authors': 'Philipp Seidl', 'comments': ' 13 pages + 6 pages appendix\n', 'primary-subject': 'Machine Learning (cs.LG)'}
{'title': ' Evaluation of Time Series Forecasting Models for Estimation of PM2.5  Levels in Air\n', 'authors': 'Satvik Garg', 'comments